In [1]:
from leukemia_extract import extract_data
from api_extraction import api_data_extraction
from api_transformations import process_world_bank_data
from merge import merge_dataframes
from dimensional_model_transform import process_dimensions
#from gx_validations import run_all_validations
from gx import validation_results
from load_data import export_to_postgres

In [2]:
leukemia_df = extract_data()

/home/v4lentin4/Leukemia-Cancer-Risk-ETL/airflow/functions/leukemia_extract.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query, con=conn.connection)


In [3]:
api_df = api_data_extraction()

File /home/v4lentin4/Leukemia-Cancer-Risk-ETL/data/world_bank_data.csv already exists. Loading data from the file.


In [4]:
api_df = process_world_bank_data(api_df)
api_df.head()

,country,co2_emissions_per_capita,nuclear_energy_pct,agri_employment_pct,fertilizer_consumption,gdp_per_capita,pm25_pollution,undernourishment_rate,alcohol_consumption_liters
0,argentina,4.035680,5.484569,0.612884,255.653331,14187.482725,14.624551,3.4,7.69
1,australia,14.014672,0.000000,2.147565,252.374500,64820.911789,7.387603,2.5,10.57
2,brazil,2.271017,2.049616,8.202819,391.647128,10294.866681,14.927492,3.7,8.36
3,canada,14.340252,14.301098,1.271544,24.823229,53431.185706,7.795220,2.5,9.92
4,china,9.399266,4.667542,22.333710,98.189967,12614.061742,50.299193,2.5,5.26


In [5]:
df = merge_dataframes(leukemia_df, api_df)
df.head()

,id,age,gender,country,wbc_count,rbc_count,platelet_count,hemoglobin_level,bone_marrow_blasts,genetic_mutation,...,leukemia_status,living_status,co2_emissions_per_capita,nuclear_energy_pct,agri_employment_pct,fertilizer_consumption,gdp_per_capita,pm25_pollution,undernourishment_rate,alcohol_consumption_liters
0,1,52,Male,china,2698,5.36,262493,12.2,72,1,...,0,1,9.399266,4.667542,22.33371,98.189967,12614.061742,50.299193,2.5,5.26
1,2,15,Female,china,4857,4.81,277877,11.9,97,1,...,1,0,9.399266,4.667542,22.33371,98.189967,12614.061742,50.299193,2.5,5.26
2,12,3,Male,china,6973,5.60,295077,11.9,19,0,...,0,1,9.399266,4.667542,22.33371,98.189967,12614.061742,50.299193,2.5,5.26
3,56,60,Male,china,8233,5.50,194008,16.2,79,0,...,1,0,9.399266,4.667542,22.33371,98.189967,12614.061742,50.299193,2.5,5.26
4,62,78,Male,china,7685,5.09,354128,13.0,81,1,...,0,0,9.399266,4.667542,22.33371,98.189967,12614.061742,50.299193,2.5,5.26


In [6]:
df = process_dimensions(df)

In [8]:
Dim_PatientInfo = df['Dim_PatientInfo']
Dim_MedicalHistory = df['Dim_MedicalHistory']
Dim_Region= df['Dim_Region']
Fact_Leukemia = df['Fact_Leukemia']

In [9]:
Dim_MedicalHistory.head()

,medical_history_id,genetic_mutation,family_history,smoking_status,alcohol_consumption,radiation_exposure,infection_history,chronic_illness,immune_disorders
0,1,1,0,1,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0
3,4,0,0,1,0,0,0,0,0
4,5,1,1,1,0,0,1,1,1


##### Trying gx

In [14]:
import great_expectations as gx 

context = gx.get_context(mode = "ephemeral")

In [15]:
print(type(context).__name__)

EphemeralDataContext


In [ ]:
# data



In [17]:
Dim_PatientInfo.head()

,patient_id,age,gender,socioeconomic_status,urban_rural,medical_history_id
0,1,52,Male,Low,Rural,1
1,2,15,Female,Low,Urban,2
2,12,3,Male,Medium,Urban,3
3,56,60,Male,Low,Rural,4
4,62,78,Male,High,Rural,5


In [18]:
# create a data source
data_source_name = "Dim_PatientInfo"
data_source = context.data_sources.add_pandas(name = data_source_name)

In [19]:
# data asset
data_asset_name = "patient_info_asset"
data_asset = data_source.add_dataframe_asset(name = data_asset_name)

In [ ]:
# Batch definition
batch_definition_name = "my_batch_definition"
batch_definition = data_asset.add_batch_definition_whole_dataframe(
    batch_definition_name
)

In [21]:
expectation = gx.expectations.ExpectColumnValuesToBeBetween(
    column="age", max_value=110, min_value=0
)

In [22]:
suite_name = "my_expectation_suite"
suite = gx.ExpectationSuite(name=suite_name)

In [23]:
suite = context.suites.add(suite)

In [24]:
suite.add_expectation(expectation)

ExpectColumnValuesToBeBetween(id='97a43d4e-e901-4760-839e-100c721e473b', meta=None, notes=None, result_format=<ResultFormat.BASIC: 'BASIC'>, description=None, catch_exceptions=True, rendered_content=None, windows=None, batch_id=None, column='age', mostly=1, row_condition=None, condition_parser=None, min_value=0.0, max_value=110.0, strict_min=False, strict_max=False)

In [25]:
definition_name = "my_validation_definition"
validation_definition = gx.ValidationDefinition(
    data=batch_definition, suite=suite, name=definition_name
)

In [26]:
validation_definition = context.validation_definitions.add(validation_definition)

In [27]:
batch_parameters_dataframe = {"dataframe": Dim_PatientInfo}

In [28]:
validation_results = validation_definition.run(batch_parameters=batch_parameters_dataframe, result_format="BOOLEAN_ONLY")

Calculating Metrics: 100%|██████████| 7/7 [00:00<00:00, 186.68it/s] 


In [29]:
print(validation_results)

{
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_be_between",
        "kwargs": {
          "batch_id": "Dim_PatientInfo-patient_info_asset",
          "column": "age",
          "min_value": 0.0,
          "max_value": 110.0
        },
        "meta": {},
        "id": "97a43d4e-e901-4760-839e-100c721e473b"
      },
      "result": {},
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    }
  ],
  "suite_name": "my_expectation_suite",
  "suite_parameters": {},
  "statistics": {
    "evaluated_expectations": 1,
    "successful_expectations": 1,
    "unsuccessful_expectations": 0,
    "success_percent": 100.0
  },
  "meta": {
    "great_expectations_version": "1.4.3",
    "batch_spec": {
      "batch_data": "PandasDataFrame"
    },
    "batch_markers": {
      "ge_load_time": "20250511T173319.65

### Keep going

In [7]:
validation_results = validation_results(df)
print(validation_results)

Calculating Metrics: 100%|██████████| 58/58 [00:00<00:00, 498.75it/s]


Validation results for Dim_PatientInfo:
{
  "success": true,
  "results": [
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_to_exist",
        "kwargs": {
          "batch_id": "pandas_source-dataframe_asset",
          "column": "patient_id"
        },
        "meta": {},
        "id": "374fa5d6-accd-480d-861a-dfac8057c0f9"
      },
      "result": {},
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_traceback": null,
        "exception_message": null
      }
    },
    {
      "success": true,
      "expectation_config": {
        "type": "expect_column_values_to_not_be_null",
        "kwargs": {
          "batch_id": "pandas_source-dataframe_asset",
          "column": "patient_id"
        },
        "meta": {},
        "id": "338e85a5-5f3b-43a7-82f8-53af11a0e8be"
      },
      "result": {},
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_trac

In [15]:
#export_to_postgres(df)

In [16]:
import great_expectations as gx

print(gx.__version__)

1.4.3
